# MBSP Cosine Similarity Explorer

This notebook demonstrates interactive cosine similarity search across Sentinel-2 TOA bands. Select a pixel on the MBSP fractional image to compare all other pixels in the scene.

In [ ]:
# Interactive cosine-similarity search across Sentinel-2 TOA bands
# ----------------------------------------------------------------
# DEBUG VERSION 2025-06-10
#
# Added **print statements** inside the click handler to dump:
#   • the reference spectrum values
#   • dot, mag1, mag2, similarity for the clicked pixel
#   • similarity min/max over the 2-km buffer region
#
# No other logic changed.

import datetime as dt
import ee
import geemap
import ipywidgets as widgets

ee.Authenticate()
ee.Initialize()


# ----------------------------------------------------------------
# Helpers
# ----------------------------------------------------------------
def mask_s2_clouds(image: ee.Image) -> ee.Image:
    """Mask clouds using the QA60 band and scale reflectance to [0,1]."""
    qa = image.select("QA60")
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return (
        image.updateMask(mask)
        .divide(10000)
        .copyProperties(image, image.propertyNames())
    )


def mbsp_fractional_image(image: ee.Image, region: ee.Geometry) -> ee.Image:
    """Compute the MBSP fractional image used for pixel selection."""
    num_img = image.select("B11").multiply(image.select("B12"))
    den_img = image.select("B12").multiply(image.select("B12"))
    num_sum = num_img.reduceRegion(
        reducer=ee.Reducer.sum(), geometry=region, scale=20, bestEffort=True
    )
    den_sum = den_img.reduceRegion(
        reducer=ee.Reducer.sum(), geometry=region, scale=20, bestEffort=True
    )
    slope = ee.Number(num_sum.get("B11")).divide(ee.Number(den_sum.get("B12")))
    mbsp = (
        image.select("B12")
        .multiply(slope)
        .subtract(image.select("B11"))
        .divide(image.select("B11"))
        .rename("R")
    )
    return mbsp.set({"slope": slope})


# Sentinel-2 bands (drop B10)
BANDS = [
    "B1",
    "B2",
    "B3",
    "B4",
    "B5",
    "B6",
    "B7",
    "B8",
    "B8A",
    "B9",
    "B11",
    "B12",
]


# ----------------------------------------------------------------
# Cosine similarity
# ----------------------------------------------------------------
def cosine_similarity_image(image: ee.Image, ref_feat: ee.Feature) -> ee.Image:
    """Return an image whose single band is cosine similarity to the reference pixel."""
    ref_vals = ee.Image.constant([ee.Number(ref_feat.get(b)) for b in BANDS]).rename(
        BANDS
    )
    proj20 = image.select("B11").projection()  # 20-m grid
    img = image.select(BANDS).resample("bilinear").reproject(proj20)
    ref = ref_vals.reproject(proj20)

    dot = img.multiply(ref).reduce(ee.Reducer.sum()).rename("dot")
    mag1 = img.pow(2).reduce(ee.Reducer.sum()).sqrt().rename("mag1")
    mag2 = ref.pow(2).reduce(ee.Reducer.sum()).sqrt().rename("mag2")

    similarity = dot.divide(mag1.multiply(mag2)).rename("similarity")
    similarity = similarity.updateMask(img.mask().reduce(ee.Reducer.anyNonZero()))
    return similarity


# ----------------------------------------------------------------
# Data selection
# ----------------------------------------------------------------
lat, lon = 31.6585, 5.9053
start = dt.date(2019, 10, 1)
end = dt.date(2019, 10, 15)
point = ee.Geometry.Point(lon, lat)

collection = (
    ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
    .filterDate(str(start), str(end))
    .filterBounds(point)
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
    .sort("system:time_start")
    .map(mask_s2_clouds)
)
images = collection.toList(collection.size())
count = images.size().getInfo()
print(f"Found {count} images")

# ----------------------------------------------------------------
# Interactive map
# ----------------------------------------------------------------
if count:
    region = point.buffer(1000).bounds()
    img = ee.Image(images.get(0))
    date = ee.Date(img.get("system:time_start")).format("YYYY-MM-dd").getInfo()

    frac = mbsp_fractional_image(img, region)

    m = geemap.Map(center=(lat, lon), zoom=12)
    m.addLayer(img.select(["B4", "B3", "B2"]), {"min": 0, "max": 0.3}, "RGB", False)
    m.addLayer(
        frac,
        {"min": -0.05, "max": 0.05, "palette": ["blue", "white", "red"]},
        "Fractional",
        True,
    )
    styled_pt = ee.FeatureCollection([point]).style(
        **{
            "color": "green",  # outline & fill
            "fillColor": None,  # no fill
            "pointSize": 8,  # pixel radius of the dot
        }
    )
    frac_layer = m.layers[-1]
    m.sim_layer = None  # will hold the current similarity layer

    # ------------------------------------------------------------
    # Click handler with DEBUG prints
    # ------------------------------------------------------------
    def handle_click(**kwargs):
        if kwargs.get("type") != "click":
            return

        latc, lonc = kwargs.get("coordinates")
        pt = ee.Geometry.Point(lonc, latc)

        # Sample reference pixel
        sample_fc = img.select(BANDS).sample(pt, scale=20, numPixels=1, tileScale=1)
        if sample_fc.size().getInfo() == 0:
            print("No valid data at clicked location.")
            return
        ref_feat = ee.Feature(sample_fc.first())

        # DEBUG: print reference spectrum
        print("\n--- DEBUG ---")
        print("Reference spectrum (reflectance):")
        print(ref_feat.toDictionary().getInfo())

        # Build similarity layer
        sim = cosine_similarity_image(img, ref_feat)

        # DEBUG: print dot/mag/sim values at click location
        proj20 = img.select("B11").projection()
        ref_vals = (
            ee.Image.constant([ee.Number(ref_feat.get(b)) for b in BANDS])
            .rename(BANDS)
            .reproject(proj20)
        )
        img20 = img.select(BANDS).resample("bilinear").reproject(proj20)
        dot_img = img20.multiply(ref_vals).reduce(ee.Reducer.sum()).rename("dot")
        mag1_img = img20.pow(2).reduce(ee.Reducer.sum()).sqrt().rename("mag1")
        mag2_img = ref_vals.pow(2).reduce(ee.Reducer.sum()).sqrt().rename("mag2")

        dot_val = dot_img.sample(pt, scale=20).first().get("dot").getInfo()
        mag1_val = mag1_img.sample(pt, scale=20).first().get("mag1").getInfo()
        mag2_val = mag2_img.sample(pt, scale=20).first().get("mag2").getInfo()
        sim_val = sim.sample(pt, scale=20).first().get("similarity").getInfo()
        print(f"dot={dot_val}, mag1={mag1_val}, mag2={mag2_val}, sim={sim_val}")

        # DEBUG: similarity min/max in 2-km region
        stats = sim.reduceRegion(
            reducer=ee.Reducer.minMax(), geometry=region, scale=20, bestEffort=True
        ).getInfo()
        print("Similarity min/max in region:", stats)
        print("--- END DEBUG ---\n")

        # Remove previous similarity layer before adding new one
        if m.sim_layer is not None:
            m.remove_layer(m.sim_layer)

        m.addLayer(
            sim,
            {"min": 0.8, "max": 1, "palette": ["white", "green"]},
            "Similarity",
            False,
        )
        m.sim_layer = m.layers[-1]

    m.on_interaction(handle_click)
    display(m)

    # ------------------------------------------------------------
    # Toggle buttons to show/hide layers
    # ------------------------------------------------------------
    toggle = widgets.ToggleButtons(
        options=["Fractional", "Similarity"], description="View:"
    )

    def switch(change):
        if m.sim_layer is None:
            return
        frac_layer.visible = change["new"] == "Fractional"
        m.sim_layer.visible = change["new"] == "Similarity"

    toggle.observe(switch, "value")
    display(toggle)
